## This is a notebook to archive all your Tableau Server Content to a Remote File Store

First up, install Server Client and other stuff

In [ ]:
!pip install tableauserverclient
!pip install requests
!pip install slacker

This section below is for dealing with HTTPS 'Trust' issues

In [ ]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
import os
import errno
from slacker import Slacker

This is for Slack Integration and creating our multithreaded pool

In [ ]:
import tableauserverclient as TSC
slack = Slacker(token)
request_options = TSC.RequestOptions(pagesize=1000)

Next up, configure your login

In [ ]:
tableau_auth = TSC.TableauAuth(username,password)
server = TSC.Server('url')
server.add_http_options({'verify':False})

Now login, and gather all the information you need about sites.
*Remember, Tableau will log you into the default site, and the server client doesn't have the **Site Switch** method yet*

In [ ]:
server.auth.sign_in(tableau_auth)

Create an array for Sites, populate it with Content URLs

In [ ]:
sites = []
for site in TSC.Pager(server.sites, request_options):
    sites.append({'url':site.content_url,'name':site.name})
#This is how you access the individual attributes in the dictionary
print sites[0]['name']

Helper function for creating folders

In [ ]:
def mkdir_p(path):
    if not os.path.exists(path):
        os.mkdir(path)

Now we are going to create a function that will get all the workbooks for the site

In [ ]:
def getWorkbooks():
    workbooks = []
    for wb in TSC.Pager(server.workbooks, request_options):
        workbooks.append({'id':wb.id, 'name':wb.name,'project':wb.project_name})
    return workbooks

Let's get all the workbooks and then take a look at the first one

In [ ]:
serverWorkbooks = getWorkbooks()
#Get the first one to make sure things look ok
print len(serverWorkbooks)
print serverWorkbooks[0]